<a href="https://colab.research.google.com/github/PajrulPalah/project_internship_indekstat/blob/main/Analisa_Data_Magang_Menggunakan_Query_SQL_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modul Untuk Menganlisis Data Menggunakan Query di Postgree SQL ke-3**

### Created  by: Muhammad Pajrul Palah
### Rabu, 08 November 2023


##**Silakan Untuk Dicopy Query SQL Berikut**

```(r)
-- MENGETAHUI DUKUNGAN PARTAI TERHADAP KANDIDAT PRESIDEN PADA PEMILU DI TAHUN 2019

with RankedPartai as (
select
	subquery.nm_dapil,
	subquery.nm_kota,
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.perolehan_suara_perdapil,
	(subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_dapil
order by
	subquery.perolehan_suara_perdapil desc) as rank_partai
from
	(
	select
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai,
		SUM(perolehan) as perolehan_suara_perdapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai
        ) subquery
join (
	select
		nm_dapil,
		SUM(perolehan) as total_perolehan_dapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil
        ) total on
	subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
select
	nm_dapil,
	COUNT(distinct nm_kec) as total_kecamatan,
	COUNT(distinct nm_desa) as total_desa
from
	data_magang_dprd_kab
group by
	nm_dapil
)
SELECT
    RankedPartai.*,
    tkd.total_kecamatan,
    tkd.total_desa,
    data_magang_dprd_kab.nm_desa,
    data_magang_dprd_kab.nm_kec,
    data_magang_dprd_kab.perolehan,
    ppp.mask_presiden AS dukungan_presiden
FROM
    RankedPartai
JOIN TotalKecamatanDesa tkd ON RankedPartai.nm_dapil = tkd.nm_dapil
JOIN data_magang_dprd_kab ON RankedPartai.nm_dapil = data_magang_dprd_kab.nm_dapil
    AND RankedPartai.mask_partai = data_magang_dprd_kab.mask_partai
JOIN public.partai_pdk_presiden ppp  ON RankedPartai.mask_partai = ppp.mask_partai
WHERE
    persentase_keterlibatan_partai >= 20
    AND data_magang_dprd_kab.nm_prov IN (
        'KALIMANTAN UTARA',
        'SULAWESI TENGAH',
        'SULAWESI SELATAN',
        'SULAWESI UTARA',
        'SULAWESI TENGGARA',
        'MALUKU UTARA'
    )
ORDER BY
    nm_dapil,
    nm_kota,
    rank_partai;
   
 -- MENGETAHUI TOTAL SUARA DI SETIAP PROVINSI

select
	nm_prov,
	SUM(perolehan) as total_perolehan_suara
from
	public.data_magang_dprd_kab
group by
	nm_prov
order by
	total_perolehan_suara desc;



-- UNTUK MENGETAHUI SUARA CAPRES DIMASING-MASING PROVINSI
with RankedPartai as (
select
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.total_perolehan_suara,
	(subquery.total_perolehan_suara * 100.0) / total.total_perolehan_prov as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_prov
order by
	subquery.total_perolehan_suara desc) as rank_partai
from
	(
	select
		nm_prov,
		mask_partai,
		SUM(perolehan) as total_perolehan_suara
	from
		data_magang_dprd_kab
	group by
		nm_prov,
		mask_partai
    ) subquery
join (
	select
		nm_prov,
		SUM(perolehan) as total_perolehan_prov
	from
		data_magang_dprd_kab
	group by
		nm_prov
    ) total on
	subquery.nm_prov = total.nm_prov
)
select
	RankedPartai.*,
	ppp.mask_presiden as dukungan_presiden
from
	RankedPartai
join public.partai_pdk_presiden ppp on
	RankedPartai.mask_partai = ppp.mask_partai
order by
	nm_prov,
	rank_partai;

   
-- MENGECHECK APAKAH TOTAL SUARA SAMA DENGAN DIBAGI MENJADI KE MASING-MASING PROVINSI
select sum(perolehan) as total_suara
from public.data_magang_dprd_kab;

select distinct mask_partai
from public.data_magang_dprd_kab;


-- MEMFILTER HANYA PARTAI YANG MEMILIKI NILAI PERSENTASE DI ATAS ATAU SAMA DENGAN 20 % PERDAPIL MASING-MASING
with RankedPartai as (
select
	subquery.nm_dapil,
	subquery.nm_kota,
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.perolehan_suara_perdapil,
	(subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_dapil
order by
	subquery.perolehan_suara_perdapil desc) as rank_partai
from
	(
	select
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai,
		SUM(perolehan) as perolehan_suara_perdapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai
    ) subquery
join (
	select
		nm_dapil,
		SUM(perolehan) as total_perolehan_dapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil
    ) total on
	subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
select
	nm_dapil,
	COUNT(distinct nm_kec) as total_kecamatan,
	COUNT(distinct nm_desa) as total_desa
from
	data_magang_dprd_kab
group by
	nm_dapil
)
select
	RankedPartai.*,
	tkd.total_kecamatan,
	tkd.total_desa,
	ppp.mask_presiden as dukungan_presiden
from
	RankedPartai
join TotalKecamatanDesa tkd on
	RankedPartai.nm_dapil = tkd.nm_dapil
join public.partai_pdk_presiden ppp on
	RankedPartai.mask_partai = ppp.mask_partai
where
	persentase_keterlibatan_partai >= 20
order by
	nm_dapil,
	nm_kota,
	rank_partai;

-- TANPA KONDISI WHERE
with RankedPartai as (
select
	subquery.nm_dapil,
	subquery.nm_kota,
	subquery.nm_prov,
	subquery.mask_partai,
	subquery.perolehan_suara_perdapil,
	(subquery.perolehan_suara_perdapil * 100.0) / total.total_perolehan_dapil as persentase_keterlibatan_partai,
	rank() over (partition by subquery.nm_dapil
order by
	subquery.perolehan_suara_perdapil desc) as rank_partai
from
	(
	select
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai,
		SUM(perolehan) as perolehan_suara_perdapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil,
		nm_kota,
		nm_prov,
		mask_partai
    ) subquery
join (
	select
		nm_dapil,
		SUM(perolehan) as total_perolehan_dapil
	from
		data_magang_dprd_kab
	group by
		nm_dapil
    ) total on
	subquery.nm_dapil = total.nm_dapil
),
TotalKecamatanDesa as (
select
	nm_dapil,
	COUNT(distinct nm_kec) as total_kecamatan,
	COUNT(distinct nm_desa) as total_desa
from
	data_magang_dprd_kab
group by
	nm_dapil
)
select
	RankedPartai.*,
	tkd.total_kecamatan,
	tkd.total_desa,
	ppp.mask_presiden as dukungan_presiden
from
	RankedPartai
join TotalKecamatanDesa tkd on
	RankedPartai.nm_dapil = tkd.nm_dapil
join public.partai_pdk_presiden ppp on
	RankedPartai.mask_partai = ppp.mask_partai
order by
	nm_dapil,
	nm_kota,
	rank_partai;

-- UNTUK MEMILIH KOLOM TERTENTU SAJA DAN MENGHILANGKAN VALUE 0 PADA KOLOM PEROLEHAN
select *
from data_magang_dprd_kab dmdk
where perolehan > 0;

```